# Session 4: Experimental Design II
## Statistical Analysis and Causal Testing

**Production LLM Deployment: Risk Characterization Before Failure**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Javihaus/Production_LLM_Deployment/blob/main/sessions/session_04_experimental_design_ii/notebook.ipynb)

---

**Learning Objectives:**
1. Detect bimodal distributions in model performance
2. Quantify prompt brittleness with statistical significance
3. Measure false positive/negative bias rates
4. Understand causal intervention approaches

## Setup

In [ ]:
!pip install -q anthropic numpy pandas matplotlib seaborn scipy statsmodels

import anthropic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from typing import List, Dict, Tuple
import json
import time

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

try:
    from google.colab import userdata
    api_key = userdata.get('ANTHROPIC_API_KEY')
except:
    import os
    api_key = os.environ.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(api_key=api_key)
print("Setup complete!")

## Part 1: Detecting Bimodal Distributions

Model performance on specific tasks often shows bimodal rather than normal distributions.

In [ ]:
# Simulated model performance data (based on real experiments)
# Each value is accuracy (%) for a model on temporal reasoning tasks

model_accuracies = {
    "Phi-2 (2.7B)": 62.5,
    "Phi-3 (3.8B)": 62.5,
    "Mistral (7B)": 62.5,
    "Llama-2 (7B)": 37.5,
    "Gemma (7B)": 50.0,
    "Llama-3 (8B)": 75.0,
    "Qwen (7B)": 37.5,
    "DeepSeek (7B)": 62.5,
}

accuracies = list(model_accuracies.values())

# Visualize distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(accuracies, bins=6, edgecolor='black', alpha=0.7)
axes[0].axvline(np.mean(accuracies), color='red', linestyle='--', label=f'Mean: {np.mean(accuracies):.1f}%')
axes[0].set_xlabel('Accuracy (%)')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Model Accuracies\non Temporal Reasoning')
axes[0].legend()

# Bar chart by model
models = list(model_accuracies.keys())
accs = list(model_accuracies.values())
colors = ['green' if a >= 60 else 'orange' if a >= 45 else 'red' for a in accs]
axes[1].barh(models, accs, color=colors, alpha=0.7)
axes[1].axvline(50, color='gray', linestyle='--', label='Random chance')
axes[1].set_xlabel('Accuracy (%)')
axes[1].set_title('Performance by Model')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nDistribution Statistics:")
print(f"  Mean: {np.mean(accuracies):.1f}%")
print(f"  Std: {np.std(accuracies):.1f}%")
print(f"  Min: {np.min(accuracies):.1f}%")
print(f"  Max: {np.max(accuracies):.1f}%")
print(f"\nNote: Bimodal clustering around ~37.5% and ~62.5%")

In [ ]:
def test_bimodality(data: List[float]) -> Dict:
    """Test for bimodality using dip test and coefficient."""
    
    # Bimodality coefficient (Sarle's)
    n = len(data)
    skewness = stats.skew(data)
    kurtosis = stats.kurtosis(data)
    
    # Bimodality coefficient formula
    bc = (skewness**2 + 1) / (kurtosis + 3 * ((n-1)**2) / ((n-2)*(n-3)))
    
    # BC > 0.555 suggests bimodality
    is_bimodal = bc > 0.555
    
    return {
        "bimodality_coefficient": bc,
        "is_bimodal": is_bimodal,
        "skewness": skewness,
        "kurtosis": kurtosis,
        "interpretation": "Bimodal distribution detected" if is_bimodal else "Unimodal distribution"
    }

bimodality_result = test_bimodality(accuracies)
print("Bimodality Test Results:")
for k, v in bimodality_result.items():
    print(f"  {k}: {v}")

## Part 2: Quantifying Brittleness

In [ ]:
# Simulated brittleness data: accuracy by prompt format
brittleness_data = {
    "natural": [75, 62.5, 75, 62.5, 50],
    "clinical": [50, 37.5, 50, 50, 25],
    "json": [25, 12.5, 25, 25, 0],
    "conversational": [62.5, 50, 62.5, 50, 37.5],
    "formal": [62.5, 50, 75, 62.5, 50]
}

df_brittleness = pd.DataFrame(brittleness_data)
df_brittleness.index = [f"Model {i+1}" for i in range(len(df_brittleness))]

# Calculate brittleness metrics
df_brittleness['max_accuracy'] = df_brittleness.max(axis=1)
df_brittleness['min_accuracy'] = df_brittleness.min(axis=1)
df_brittleness['brittleness_pp'] = df_brittleness['max_accuracy'] - df_brittleness['min_accuracy']

print("=" * 60)
print("BRITTLENESS ANALYSIS")
print("=" * 60)
print(df_brittleness.to_string())

print(f"\nMean brittleness: {df_brittleness['brittleness_pp'].mean():.1f} percentage points")
print(f"Max brittleness: {df_brittleness['brittleness_pp'].max():.1f} percentage points")

In [ ]:
# Visualize brittleness
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Heatmap of accuracies by format
format_cols = ['natural', 'clinical', 'json', 'conversational', 'formal']
sns.heatmap(
    df_brittleness[format_cols], 
    annot=True, 
    fmt='.1f',
    cmap='RdYlGn',
    ax=axes[0],
    vmin=0,
    vmax=100
)
axes[0].set_title('Accuracy by Prompt Format (%)')

# Brittleness by model
axes[1].barh(
    df_brittleness.index, 
    df_brittleness['brittleness_pp'],
    color=['red' if b > 50 else 'orange' if b > 30 else 'green' for b in df_brittleness['brittleness_pp']]
)
axes[1].axvline(30, color='gray', linestyle='--', label='High threshold')
axes[1].set_xlabel('Brittleness (percentage points)')
axes[1].set_title('Brittleness by Model')
axes[1].legend()

plt.tight_layout()
plt.show()

## Part 3: Measuring Bias Rates

In [ ]:
def calculate_bias_rates(predictions: List[str], ground_truth: List[str]) -> Dict:
    """Calculate false positive and false negative rates."""
    
    # Count outcomes
    tp = sum(1 for p, g in zip(predictions, ground_truth) if p == "YES" and g == "YES")
    tn = sum(1 for p, g in zip(predictions, ground_truth) if p == "NO" and g == "NO")
    fp = sum(1 for p, g in zip(predictions, ground_truth) if p == "YES" and g == "NO")
    fn = sum(1 for p, g in zip(predictions, ground_truth) if p == "NO" and g == "YES")
    
    # Calculate rates
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
    accuracy = (tp + tn) / len(predictions)
    
    return {
        "true_positives": tp,
        "true_negatives": tn,
        "false_positives": fp,
        "false_negatives": fn,
        "false_positive_rate": fpr,
        "false_negative_rate": fnr,
        "accuracy": accuracy
    }


# Example: Action bias demonstration
# Model that always recommends action (says YES)
ground_truth = ["YES", "YES", "YES", "YES", "NO", "NO", "NO", "NO"]
biased_predictions = ["YES", "YES", "YES", "YES", "YES", "YES", "YES", "YES"]  # Always YES
unbiased_predictions = ["YES", "YES", "NO", "YES", "NO", "NO", "YES", "NO"]  # Mixed

biased_results = calculate_bias_rates(biased_predictions, ground_truth)
unbiased_results = calculate_bias_rates(unbiased_predictions, ground_truth)

print("=" * 60)
print("BIAS RATE ANALYSIS")
print("=" * 60)

print("\nBiased Model (always says YES):")
print(f"  Accuracy: {biased_results['accuracy']:.1%}")
print(f"  False Positive Rate: {biased_results['false_positive_rate']:.1%}")
print(f"  False Negative Rate: {biased_results['false_negative_rate']:.1%}")

print("\nUnbiased Model (mixed responses):")
print(f"  Accuracy: {unbiased_results['accuracy']:.1%}")
print(f"  False Positive Rate: {unbiased_results['false_positive_rate']:.1%}")
print(f"  False Negative Rate: {unbiased_results['false_negative_rate']:.1%}")

print("\n" + "=" * 60)
print("Key Insight: 100% FPR means the model never correctly identifies")
print("'unsafe' scenarios - critical for high-stakes applications!")
print("=" * 60)

In [ ]:
# Visualize confusion matrix
def plot_confusion_matrix(results: Dict, title: str):
    """Plot confusion matrix."""
    cm = np.array([
        [results['true_negatives'], results['false_positives']],
        [results['false_negatives'], results['true_positives']]
    ])
    
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d',
        cmap='Blues',
        xticklabels=['Predicted NO', 'Predicted YES'],
        yticklabels=['Actual NO', 'Actual YES'],
        ax=ax
    )
    ax.set_title(f'{title}\nAccuracy: {results["accuracy"]:.1%}, FPR: {results["false_positive_rate"]:.1%}')
    plt.tight_layout()
    return fig

plot_confusion_matrix(biased_results, "Biased Model (Always YES)")
plt.show()

## Part 4: Statistical Significance Testing

In [ ]:
def test_format_difference(format_a_results: List[bool], format_b_results: List[bool]) -> Dict:
    """Test if performance difference between formats is significant."""
    
    # McNemar's test for paired nominal data
    # Count disagreements
    b = sum(1 for a, b in zip(format_a_results, format_b_results) if a and not b)  # A correct, B wrong
    c = sum(1 for a, b in zip(format_a_results, format_b_results) if not a and b)  # A wrong, B correct
    
    # McNemar's statistic
    if b + c > 0:
        statistic = (abs(b - c) - 1)**2 / (b + c)
        p_value = 1 - stats.chi2.cdf(statistic, df=1)
    else:
        statistic = 0
        p_value = 1.0
    
    accuracy_a = sum(format_a_results) / len(format_a_results)
    accuracy_b = sum(format_b_results) / len(format_b_results)
    
    return {
        "accuracy_format_a": accuracy_a,
        "accuracy_format_b": accuracy_b,
        "difference_pp": (accuracy_a - accuracy_b) * 100,
        "mcnemar_statistic": statistic,
        "p_value": p_value,
        "significant": p_value < 0.05
    }


# Example: Compare natural vs JSON format
# Simulated paired results
natural_results = [True, True, False, True, True, True, False, True]  # 75% accuracy
json_results = [False, True, False, False, False, True, False, False]  # 25% accuracy

significance = test_format_difference(natural_results, json_results)

print("=" * 60)
print("STATISTICAL SIGNIFICANCE: Natural vs JSON Format")
print("=" * 60)
print(f"Natural format accuracy: {significance['accuracy_format_a']:.1%}")
print(f"JSON format accuracy: {significance['accuracy_format_b']:.1%}")
print(f"Difference: {significance['difference_pp']:.1f} percentage points")
print(f"McNemar statistic: {significance['mcnemar_statistic']:.2f}")
print(f"P-value: {significance['p_value']:.4f}")
print(f"Significant (p<0.05): {'YES' if significance['significant'] else 'NO'}")

## Part 5: Exercise - Analyze Your Results

In [ ]:
# YOUR EXERCISE: Analyze your test results

# Replace with your actual data
my_predictions = ["YES", "NO", "YES", "YES", "NO", "NO", "YES", "NO"]
my_ground_truth = ["YES", "YES", "NO", "YES", "NO", "NO", "NO", "NO"]

my_results = calculate_bias_rates(my_predictions, my_ground_truth)

print("YOUR RESULTS:")
print(f"  Accuracy: {my_results['accuracy']:.1%}")
print(f"  False Positive Rate: {my_results['false_positive_rate']:.1%}")
print(f"  False Negative Rate: {my_results['false_negative_rate']:.1%}")

## Key Takeaways

1. **Bimodal distributions are diagnostic.** Models either "get it" or don't - no continuous scaling.

2. **Brittleness reveals pattern matching.** >30pp accuracy change across formats = not robust understanding.

3. **Bias rates matter more than accuracy.** A 100% FPR is catastrophic even with 50% overall accuracy.

4. **Test for significance.** Small sample differences may not be meaningful.

5. **Visualize everything.** Patterns become obvious in good visualizations.

---

**Homework:** Complete statistical analysis of your test results.

**Next Session:** Failure Mode I—Temporal Constraint Processing